In [8]:
%useLatestDescriptors
%use lets-plot

In [9]:
import java.io.File

val algResultsDir =
    "D:${File.separator}" +
            "EDOLAB-MATLAB${File.separator}" +
            "Results${File.separator}" +
            "Comparison${File.separator}" +
            "WBO25${File.separator}" +
            "MPB_Peaks10_ChangeFrequency5000_D5_ShiftSeverity1_Environments100"

In [10]:
import org.um.feri.ears.algorithms.DummyAlgorithm
import org.um.feri.ears.algorithms.NumberAlgorithm

val players = arrayListOf<NumberAlgorithm>(
    DummyAlgorithm("ACFPSO", algResultsDir),
    DummyAlgorithm("AMPDE", algResultsDir),
    DummyAlgorithm("CPSO", algResultsDir),
    //DummyAlgorithm("mjDE", algResultsDir)
)

In [11]:
import org.um.feri.ears.examples.dynopt.AlgorithmPerformance

val algPerf = players.map { AlgorithmPerformance(it.id) }.toMutableList() // Prepare objects for storing the algorithms data.

val runNumber = 31

val changeFrequency = 5000
val environmentNumber = 100
val sampleInterval = 100
val numOfProblems = (changeFrequency / sampleInterval) * environmentNumber + environmentNumber - 1

In [12]:
import org.um.feri.ears.benchmark.Benchmark
import org.um.feri.ears.examples.dynopt.MPBBenchmark
import org.um.feri.ears.statistic.rating_system.RatingType
import org.um.feri.ears.visualization.rating.RatingIntervalPlot

Benchmark.printInfo = false
//DummyAlgorithm.readFromJson = false
val displayRatingChart = false

var evaluationNumber = sampleInterval
for (i in 0 until numOfProblems) {
    // Create and execute benchmarks for every evaluation defined with sampleInterval.
    val gmpbBenchmark = MPBBenchmark(evaluationNumber).apply {
        setDisplayRatingCharts(false)
        addAlgorithms(players)
        run(runNumber)
    }

    val tournamentResults = gmpbBenchmark.tournamentResults

    //tournamentResults.players.forEach { player ->
    //    algPerf.firstOrNull { it.name == player.id }?.apply {
    //        addRating(
    //            "mpbeval0",
    //            1500.0,
    //            350.0
    //        )
    //    }
    //}

    if (displayRatingChart) {
        RatingIntervalPlot.displayChart(
            tournamentResults.players,
            RatingType.GLICKO2,
            "Rating Interval for Eval$evaluationNumber"
        )
    }

    // Store tournament results for every player (algorithm).
    tournamentResults.players.forEach { player ->
        algPerf.firstOrNull { it.name == player.id }?.apply {
            addRating(
                "mpbeval$evaluationNumber",
                player.glicko2Rating.rating,
                player.glicko2Rating.ratingDeviation
            )
        }
    }

    if (evaluationNumber % changeFrequency == 0) {
        // first evaluation after environment/problem change
        evaluationNumber++
    } else if (evaluationNumber % sampleInterval == 0) {
        // evaluation every 'sampleInterval'
        evaluationNumber += sampleInterval
    } else {
        // next evaluation after environment/problem change
        evaluationNumber = evaluationNumber + sampleInterval - 1
    }
}

TrueSkill One-On-One rating:
ACFPSO - Mean(mu)=21,6, Std-Dev(sigma)=1,48 RI=[18,64, 24,57]
AMPDE - Mean(mu)=18,01, Std-Dev(sigma)=1,13 RI=[15,74, 20,27]
CPSO - Mean(mu)=17,52, Std-Dev(sigma)=1,08 RI=[15,37, 19,68]

TrueSkill Free-For-All rating:
ACFPSO - Mean(mu)=27,82, Std-Dev(sigma)=1,48 RI=[24,85, 30,79]
AMPDE - Mean(mu)=22,37, Std-Dev(sigma)=1,16 RI=[20,05, 24,68]
CPSO - Mean(mu)=21,85, Std-Dev(sigma)=1,16 RI=[19,53, 24,16]

Glicko2 rating:
ACFPSO - Rating=1.839,7 RD=64,8 ro=0,06 RI=[1710,1, 1969,34]
AMPDE - Rating=1.338,2 RD=64,8 ro=0,06 RI=[1208,61, 1467,85]
CPSO - Rating=1.322,1 RD=64,8 ro=0,06 RI=[1192,43, 1451,67]

Game results:
ACFPSO [win=52, lose=10, draw=0]
	 Against:{CPSO=[win=26, lose=5, draw=0], AMPDE=[win=26, lose=5, draw=0]}
	 Problems:{MPBEval100=[win=52, lose=10, draw=0]}
AMPDE [win=13, lose=33, draw=16]
	 Against:{ACFPSO=[win=5, lose=26, draw=0], CPSO=[win=8, lose=7, draw=16]}
	 Problems:{MPBEval100=[win=13, lose=33, draw=16]}
CPSO [win=12, lose=34, draw=16]
	 Agai

In [13]:
fun generateSequence(changeFrequency: Int, environmentNumber: Int, sampleInterval: Int = 1000): List<Int> {
    val result = mutableListOf<Int>()
    // generate numbers increasing by sampleInterval
    for (i in sampleInterval..changeFrequency * environmentNumber step sampleInterval) {
        result.add(i)
        if (i % changeFrequency == 0 && i != changeFrequency * environmentNumber) {
            result.add(i + 1)   // add the next (evaluation) number just after the change happens
        }
    }
    return result
}

In [16]:
import org.jetbrains.letsPlot.commons.values.Color

val numAlgorithms = algPerf.size
val xValues = generateSequence(changeFrequency, environmentNumber, sampleInterval)

var plot = letsPlot(mapOf("eval" to xValues))

for (i in 0 until numAlgorithms) {
    val yValues = algPerf[i].evalData.values.map { it.rating } // // extract the rating values for the current algorithm

    // calculate the lower and upper bounds for the rating deviations
    val lowerBounds = algPerf[i].evalData.values.map { it.rating - 2 * it.ratingDeviation }
    val upperBounds = algPerf[i].evalData.values.map { it.rating + 2 * it.ratingDeviation }

    // prepare data for the current algorithm
    val data = mapOf(
        "Number of Fitness Evaluations" to xValues,
        "Rating" to yValues,
        "lowerBounds" to lowerBounds,
        "upperBounds" to upperBounds,
        "DMA:" to List(yValues.size) { algPerf[i].name }  // label each algorithm
    )

    // add rating line to the plot (cycling through predefined colors)
    plot += geomLine(data = data) {
        x = "Number of Fitness Evaluations"
        y = "Rating"
        color = "DMA:"
    }

    // add rating deviation ribbon to the plot (using the same color with transparency)
    plot += geomRibbon(data = data, alpha = 0.2) {
        x = "Number of Fitness Evaluations"
        ymin = "lowerBounds"
        ymax = "upperBounds"
        color = "DMA:"
        fill = "DMA:"
    }

    // add points at the same locations as the line
    plot += geomPoint(data = data) {
        x = "Number of Fitness Evaluations"
        y = "Rating"
        color = "DMA:"
    }
}

val minX = 0
val maxX = 25000

val shownTicks = xValues.withIndex()
    .filter { it.value % 5000 == 0 }
    .map { it.value }

plot += scaleXContinuous(
    breaks = shownTicks,
    labels = shownTicks.map { it.toInt().toString() }, // specify the tick marks   .take(valuesToShow)
    limits = minX to maxX
)

val verticalLines = (changeFrequency..changeFrequency * environmentNumber step changeFrequency).toList()
verticalLines.forEach { evalPoint ->
    plot += geomVLine(xintercept = evalPoint, color = "grey", linetype = "dashed") {
        // you can customize the appearance here
    }
}

// val width = 1000; val height = 750   // article dimensions
val width = 2000; val height = 750  // presentation dimensions

plot += ggsize(width, height)
plot += theme(
    title = elementText(face = "bold"), // 'title' applies to plot's title, subtitle, caption
    axisTitleX = elementText(size = 24),
    axisTextX = elementText(size = 20, angle = 0),
    axisTitleY = elementText(size = 24),
    axisTextY = elementText(size = 20),
    legendTitle = elementText(size = 24),
    legendText = elementText(size = 20)
).legendPositionTop()
plot += ggtb()  // enable Zoom and Pan interactivity

plot.show()